Thoughts Rules Engine
====================

Thoughts is a lightweight rules engine.

## Import and Start a New Engine

Start a new engine.

In [1]:
# import the thoughts rules engine
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
from thoughts.rules_engine import RulesEngine
 
engine = RulesEngine()

## Define Your Rules

In [2]:
rule = {"#when": "hello",
        "#then": {"#output": "hello, world!"}}

## Load Your Rules into the Engine

load_rules_from_list(rules, name=None) - Loads rules into memory directly from a list.

In [3]:
engine.add_rule(rule)

## Run Assertions

In [4]:
result = engine.process("hello")

hello, world!


[{'#output': 'hello, world!'}]

## Load Rules from a File

load_rules_from_file(file, name=None) - Loads a .json rules file into memory.

In [ ]:
engine.load_rules_from_file("rules.json")

## Add New Rules Manually

Alternatively, you can create a manual rule without loading a file. The following adds a rule into memory. New rules are added to the default ruleset.

In [ ]:
rule = {"#when": "what time is it", 
        "#then": {"#output": "time to get a new watch"}}
        
engine.add_rule(rule)

## Define and Run Assertions

run_assert(assertion) - Evaluates the assertion against the loaded rules. Essentially, the evaluation will attempt to match the assertion against the "when" portion of all loaded rules.

Assertions will match the "when" portion of rules, based on a unification algorithm:
* Strings will match direct string matches, "when": "hello" will match "hello"
* Strings will match using variables, "when": "my dog is ?name" will match "my dog is fido"
* Dictionaries will match a dictionary, "when": {"name": "fido"} will match {"name": "fido"}

If a rule matches, then the engine will add the "then" portion of the rule to the engine's evaluation agenda, substituting any unification variables that were determined during the "when" matching stage into the "then" items, and then evaluting them one at a time.

As each command is evaluated for assertion, the system will also substitute any values from the Context Items that are indicated in the command item.

Rules will "forward chain" - the "then" portion of rules will cause the engine to match against rules.

In [ ]:
new_rules = [

    {   "#when": "hello",
        "#then": {"user-intent": "greet"}
    },

    {   "#when": {"user-intent": "greet"},
        "#then": {"#output": "hello, world"}
    }
]

engine.load_rules_from_list(new_rules, "hello-sample")

result = engine.process("hello")

## Multiple Actions

You can have more than one command (action) in the "then" portion

In [ ]:
new_rule = { 
    "#when": "hi",
    "#then": [{"#output": "hello there"}, 
            {"#output": "nice to meet you"}]
}

engine.add_rule(new_rule)

result = engine.process("hi")

## Storing Knowledge

You can store "item" knowledge (facts).

In [ ]:
fact = {"item": "user", "name": "jeremy", "dog": "hudson"}
engine.add_rule(fact)

## Reference Stored Knowledge

You can reference the items (facts) and their properties in your rules, using $itemname.property syntax

In [ ]:
fact = {
        "#when": "what is my name", 
        "#then": [{"#output": "your name is $user.name"}]}
        
engine.add_rule(fact)

result = engine.process("what is my name")

## Sequence-based rules (chart parsing)

This is useful for natural-language type parsing where a rule needs to wait on input before firing the consequent (then) portion. In the example below, when {"cat": "art", "lemma": "the"} is asserted, the rule will match the first constituent and add the rule as an arc to the active arcs. The new arc will "wait" for another consituent with {"cat": "n", "lemma": "..."} to be asserted before matching and firing the "then" portion. Be sure to place the constituents within an array / list [] within the "when" portion.

In [ ]:
rule = {"#when": [
        {"cat" :"art", "lemma": "?det"},
        {"cat" :"n", "lemma": "?entity"}],
       "#then": 
        {"cat": "np", "entity": "?entity", "art": "?art"}
}

engine.add_rule(fact)

## Managing Active Arcs (Partial Matches in Progress)

The active rules (arcs) will remain in memory until you clear them using engine.clear_arcs(). This is useful to assert one constituent at a time into the engine to inspect the results.

In [ ]:
## Clears all active arcs (sequence rules in-progress) from memory
engine.clear_arcs()

 ## Adding your own or pip installed modules as plugins

To do this, use load_plugin() and pass in a moniker and the "dot" path of the module. This module should already have been pip installed in the environment so that the runtime can load it, or could be a standalone module in your project.

The load_plugin function Loads a plugin (Python module), which can be used in then "then "portion of rules. Whichever module you use will need to have a process function and that function will need to take two arguments - a dict and a thoughts.Context object.

Then in your rules, you can use this as a command in the "then" rules.

Your custom module has access to the Context object, which contains all of the loaded rules and items from command that ran previously.

In [ ]:
from thoughts.rules_engine import RulesEngine
engine = RulesEngine()
engine.load_plugin("#my-module", "my_module")

# my_module.py:
def process(command, context): 
    # your logic goes here
    # by convention, you can put the most relevant parameter feature into the head #my-module moniker,
    # for example text = command["#my-module"]

## Run a Console (Interactive) Mode

run_console() - Runs a console input loop. Each item entered will be passed into the engine's run_assert(assertion) function for evaluation.

Entering "#log" will display the debug log.

Entering "#items" will display the Context Items.

Entering "#clear_arcs" will clear any active sequence-based rules (arcs) from memory.

Entering "#exit" will exit the console loop. Note that the "#exit" command is also passed in as an assertion one last time, in case you want to handle the exit event first in any rules.